In [1]:
import pandas as pd 
import numpy as np
import sqlite3
conn = sqlite3.connect('CourseDataWareHouse.db')
c = conn.cursor()

In [3]:
%load_ext sql

In [5]:
%%sql
sqlite:///CourseDataWareHouse.db

'Connected: @CourseDataWareHouse.db'

In [7]:
%%sql
DROP TABLE IF EXISTS COURSES;
DROP TABLE IF EXISTS MEETINGS;
DROP TABLE IF EXISTS INSTRUCTOR;
DROP TABLE IF EXISTS TIME;
DROP TABLE IF EXISTS COURSE_FACTS;

 * sqlite:///CourseDataWareHouse.db
Done.
Done.
Done.
Done.
Done.


[]

In [8]:
%%sql
create table 'COURSES' (
    program_code VARCHAR PRIMARY KEY,
    program_name VARCHAR NOT NULL,
    course_name VARCHAR NOT NULL, 
    credits VARCHAR NOT NULL,
    prereqs VARCHAR, 
    coreqs VARCHAR,
    fees VARCHAR,
    attributes VARCHAR,
    description VARCHAR 
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [9]:
%%sql
create table 'MEETINGS' (
    ROOM_ID INTEGER PRIMARY KEY,
    ROOM_NO VARCHAR NOT NULL,
    BUILDING VARCHAR,
    STREET VARCHAR,
    AREACODE INTEGER
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [10]:
%%sql
create table 'INSTRUCTOR'(
    I_ID INTEGER PRIMARY KEY,
    NAME VARCHAR NOT NULL,
    PROGRAM_NAME VARCHAR NOT NULL,
    COURSE_TITLE VARCHAR NOT NULL,
    ROOM_NO VARCHAR NOT NULL
);


 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [11]:
%%sql
create table 'TIME'(
    TIME_ID INTEGER PRIMARY KEY,
    DAY VARCHAR,
    TERM VARCHAR NOT NULL
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [13]:
%%sql
create table 'COURSE_FACTS'(
    PROGRAM_CODE VARCHAR NOT NULL,
    CM_ID INTEGER NOT NULL,
    TERM_ID INTEGER NOT NULL,
    I_ID INTEGER NOT NULL,
    TIME_ID INTEGER NOT NULL,
    ROOM_ID INTEGER NOT NULL,
    CAP INTEGER NOT NULL,
    ACT INTEGER,
    REM INTEGER,
    MAX_CREDITS INTEGER NOT NULL,
    MAX CAP INTEGER NOT NULL,
    MIN REM INTEGER,
    FOREIGN KEY (PROGRAM_CODE) REFERENCES COURSES (PROGRAM_CODE)
    FOREIGN KEY (ROOM_ID) REFERENCES MEETINGS (ROOM_ID)
    FOREIGN KEY (I_ID) REFERENCES INSTRUCTOR (I_ID)
    FOREIGN KEY (TIME_ID) REFERENCES TIME (TIME_ID)
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [29]:
%%sql
sqlite:///CourseData.db

'Connected: @CourseData.db'

In [30]:
conn_new = sqlite3.connect('CourseData.db')
c_1 = conn.cursor()

In [32]:
import pandas as pd
courses = 'SELECT * FROM COURSE_OFFERING;'


In [33]:
courses_df = pd.read_sql(courses, conn_new)
courses_df.head()

,CO_ID,CRN,SECTION,COURSE_TITLE,CREDITS


In [21]:
courses_df['credits'].max()
courses_df['course_title'].count()

KeyError: 'credits'

In [22]:
meetings = %sql SELECT * FROM COURSE_MEETING

 * sqlite:///CourseData.db
   sqlite:///CourseDataWareHouse.db
   sqlite:///CourseDatabase.db
Done.


In [23]:
meetings_df = pd.read_sql(meetings, conn_new)
meetings_df.head()

DatabaseError: Execution failed on sql '+-------+-----+-----+-----+-----+---------+
| CM_ID | CRN | CAP | ACT | REM | SECTION |
+-------+-----+-----+-----+-----+---------+
+-------+-----+-----+-----+-----+---------+': operation parameter must be str

In [ ]:
Avg_Act = meetings_df['ACT'].mean()

In [ ]:
instructor = %sql SELECT * FROM INSTRUCTOR
instructor_df = pd.read_sql(instructor, conn_new)